# Flight Price Prediction

In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [240]:
train_df = pd.read_excel("Data_Train.xlsx")
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [241]:
# in test set one column is missing which is price because we want to predict price
test_df = pd.read_excel("Test_Set.xlsx")
test_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [242]:
# final_df contain price because it contain both train and test data by appending it
final_df = train_df.append(test_df)
final_df.head()

C:\Users\Purushoth\AppData\Local\Temp\ipykernel_3568\57058504.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = train_df.append(test_df)


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [243]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [244]:
# converting Date_of_Journey column from object to datetime format
final_df["Date_of_Journey"] = pd.to_datetime(final_df["Date_of_Journey"])
final_df.head()

C:\Users\Purushoth\AppData\Local\Temp\ipykernel_3568\1133431711.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  final_df["Date_of_Journey"] = pd.to_datetime(final_df["Date_of_Journey"])


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [245]:
final_df.iloc[8561]

Airline              Multiple carriers
Date_of_Journey    2019-09-06 00:00:00
Source                           Delhi
Destination                     Cochin
Route                  DEL → BOM → COK
Dep_Time                         10:00
Arrival_Time              01:30 10 Jun
Duration                       15h 30m
Total_Stops                     1 stop
Additional_Info                No info
Price                          13377.0
Name: 8561, dtype: object

In [246]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          13354 non-null  object        
 1   Date_of_Journey  13354 non-null  datetime64[ns]
 2   Source           13354 non-null  object        
 3   Destination      13354 non-null  object        
 4   Route            13353 non-null  object        
 5   Dep_Time         13354 non-null  object        
 6   Arrival_Time     13354 non-null  object        
 7   Duration         13354 non-null  object        
 8   Total_Stops      13353 non-null  object        
 9   Additional_Info  13354 non-null  object        
 10  Price            10683 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 1.2+ MB


In [247]:
# feature engineering
final_df["Date"] = final_df["Date_of_Journey"].dt.day
final_df["Month"] = final_df["Date_of_Journey"].dt.month
final_df["Year"] = final_df["Date_of_Journey"].dt.year

In [248]:
final_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,5,1,2019
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,6,9,2019
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,5,12,2019
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,3,1,2019


In [249]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          13354 non-null  object        
 1   Date_of_Journey  13354 non-null  datetime64[ns]
 2   Source           13354 non-null  object        
 3   Destination      13354 non-null  object        
 4   Route            13353 non-null  object        
 5   Dep_Time         13354 non-null  object        
 6   Arrival_Time     13354 non-null  object        
 7   Duration         13354 non-null  object        
 8   Total_Stops      13353 non-null  object        
 9   Additional_Info  13354 non-null  object        
 10  Price            10683 non-null  float64       
 11  Date             13354 non-null  int64         
 12  Month            13354 non-null  int64         
 13  Year             13354 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64

In [250]:
final_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,5,1,2019
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,6,9,2019
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,5,12,2019
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,3,1,2019


In [251]:
# to drop Date_of_journey
final_df.drop("Date_of_Journey",axis=1,inplace=True)
final_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,5,1,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,6,9,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,5,12,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,3,1,2019


In [252]:
# Arrival and depature time
# spliting arrival time and get time only

In [253]:
#final_df["Arrival_Time"]=final_df["Arrival_Time"].str.split(" ").str[0]
final_df["Arrival_Time"] = final_df["Arrival_Time"].apply(lambda x: x.split(" ")[0])
final_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,5,1,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882.0,6,9,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,5,12,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,3,1,2019


In [254]:
final_df.isnull().sum()

Airline               0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
Date                  0
Month                 0
Year                  0
dtype: int64

In [255]:
final_df["Dep_Time"] = pd.to_datetime(final_df["Dep_Time"])

In [256]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          13354 non-null  object        
 1   Source           13354 non-null  object        
 2   Destination      13354 non-null  object        
 3   Route            13353 non-null  object        
 4   Dep_Time         13354 non-null  datetime64[ns]
 5   Arrival_Time     13354 non-null  object        
 6   Duration         13354 non-null  object        
 7   Total_Stops      13353 non-null  object        
 8   Additional_Info  13354 non-null  object        
 9   Price            10683 non-null  float64       
 10  Date             13354 non-null  int64         
 11  Month            13354 non-null  int64         
 12  Year             13354 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 1.4+ MB


In [257]:
final_df["Dep_Hours"] = final_df["Dep_Time"].dt.hour
final_df["Dep_Minutes"] = final_df["Dep_Time"].dt.minute

In [258]:
final_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,2023-01-31 22:20:00,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2023-01-31 05:50:00,13:15,7h 25m,2 stops,No info,7662.0,5,1,2019,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2023-01-31 09:25:00,04:25,19h,2 stops,No info,13882.0,6,9,2019,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,2023-01-31 18:05:00,23:30,5h 25m,1 stop,No info,6218.0,5,12,2019,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,2023-01-31 16:50:00,21:35,4h 45m,1 stop,No info,13302.0,3,1,2019,16,50


In [259]:
# droping the column departure time
final_df.drop("Dep_Time",axis=1,inplace=True)
final_df.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662.0,5,1,2019,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25,19h,2 stops,No info,13882.0,6,9,2019,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,No info,6218.0,5,12,2019,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,No info,13302.0,3,1,2019,16,50


In [260]:
final_df["Arrival_Time"] = pd.to_datetime(final_df["Arrival_Time"])

In [261]:
final_df["Arrival_Hours"] = final_df["Arrival_Time"].dt.hour
final_df["Arrival_Minutes"] = final_df["Arrival_Time"].dt.minute

In [262]:
final_df.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,2023-01-31 01:10:00,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2023-01-31 13:15:00,7h 25m,2 stops,No info,7662.0,5,1,2019,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2023-01-31 04:25:00,19h,2 stops,No info,13882.0,6,9,2019,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,2023-01-31 23:30:00,5h 25m,1 stop,No info,6218.0,5,12,2019,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,2023-01-31 21:35:00,4h 45m,1 stop,No info,13302.0,3,1,2019,16,50,21,35


In [263]:
# droping the column arriavl time
final_df.drop("Arrival_Time",axis=1,inplace=True)
final_df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,5,1,2019,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,6,9,2019,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218.0,5,12,2019,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302.0,3,1,2019,16,50,21,35


In [264]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Duration         13354 non-null  object 
 5   Total_Stops      13353 non-null  object 
 6   Additional_Info  13354 non-null  object 
 7   Price            10683 non-null  float64
 8   Date             13354 non-null  int64  
 9   Month            13354 non-null  int64  
 10  Year             13354 non-null  int64  
 11  Dep_Hours        13354 non-null  int64  
 12  Dep_Minutes      13354 non-null  int64  
 13  Arrival_Hours    13354 non-null  int64  
 14  Arrival_Minutes  13354 non-null  int64  
dtypes: float64(1), int64(7), object(7)
memory usage: 1.6+ MB


In [265]:
final_df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,5,1,2019,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,6,9,2019,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218.0,5,12,2019,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302.0,3,1,2019,16,50,21,35


In [266]:
# considering route column
final_df["Total_Stops"].isnull().sum()

1

In [267]:
final_df["Total_Stops"].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [268]:
final_df["Total_Stops"] = final_df["Total_Stops"].map({"non-stop":0,"1 stop":1,"2 stops":2,"3 stops":3,"4 stops":4,})
final_df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0.0,No info,3897.0,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2.0,No info,7662.0,5,1,2019,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2.0,No info,13882.0,6,9,2019,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1.0,No info,6218.0,5,12,2019,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1.0,No info,13302.0,3,1,2019,16,50,21,35


In [269]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Duration         13354 non-null  object 
 5   Total_Stops      13353 non-null  float64
 6   Additional_Info  13354 non-null  object 
 7   Price            10683 non-null  float64
 8   Date             13354 non-null  int64  
 9   Month            13354 non-null  int64  
 10  Year             13354 non-null  int64  
 11  Dep_Hours        13354 non-null  int64  
 12  Dep_Minutes      13354 non-null  int64  
 13  Arrival_Hours    13354 non-null  int64  
 14  Arrival_Minutes  13354 non-null  int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 1.6+ MB


In [270]:
#final_df["Total_Stops"] = final_df["Total_Stops"].astype("int64")
# it cant convert total stops type from float to int because it contain null value if we remove null then change it
final_df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0.0,No info,3897.0,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2.0,No info,7662.0,5,1,2019,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2.0,No info,13882.0,6,9,2019,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1.0,No info,6218.0,5,12,2019,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1.0,No info,13302.0,3,1,2019,16,50,21,35


In [271]:
# total hours is null values
final_df[(final_df["Total_Stops"].isnull())]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7480.0,5,6,2019,9,45,9,25


In [272]:
final_df.drop("Route",axis=1,inplace=True)
final_df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,5,1,2019,5,50,13,15
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,6,9,2019,9,25,4,25
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,5,12,2019,18,5,23,30
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,3,1,2019,16,50,21,35


In [273]:
final_df["Additional_Info"].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [274]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Duration         13354 non-null  object 
 4   Total_Stops      13353 non-null  float64
 5   Additional_Info  13354 non-null  object 
 6   Price            10683 non-null  float64
 7   Date             13354 non-null  int64  
 8   Month            13354 non-null  int64  
 9   Year             13354 non-null  int64  
 10  Dep_Hours        13354 non-null  int64  
 11  Dep_Minutes      13354 non-null  int64  
 12  Arrival_Hours    13354 non-null  int64  
 13  Arrival_Minutes  13354 non-null  int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 1.5+ MB


In [275]:
final_df["Duration_hours"] = final_df["Duration"].apply(lambda x:x.split(" ")[0].split("h")[0])
final_df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,22,20,1,10,2
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,5,1,2019,5,50,13,15,7
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,6,9,2019,9,25,4,25,19
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,5,12,2019,18,5,23,30,5
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,3,1,2019,16,50,21,35,4


In [276]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Duration         13354 non-null  object 
 4   Total_Stops      13353 non-null  float64
 5   Additional_Info  13354 non-null  object 
 6   Price            10683 non-null  float64
 7   Date             13354 non-null  int64  
 8   Month            13354 non-null  int64  
 9   Year             13354 non-null  int64  
 10  Dep_Hours        13354 non-null  int64  
 11  Dep_Minutes      13354 non-null  int64  
 12  Arrival_Hours    13354 non-null  int64  
 13  Arrival_Minutes  13354 non-null  int64  
 14  Duration_hours   13354 non-null  object 
dtypes: float64(2), int64(7), object(6)
memory usage: 1.6+ MB


In [277]:
final_df[final_df["Duration_hours"]=="5m"]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
6474,Air India,Mumbai,Hyderabad,5m,2.0,No info,17327.0,3,6,2019,16,50,16,55,5m
2660,Air India,Mumbai,Hyderabad,5m,2.0,No info,NaN,3,12,2019,16,50,16,55,5m


In [278]:
# removing records with duration 5 minutes
final_df.drop(6474,axis=0,inplace=True)
final_df.drop(2660,axis=0,inplace=True)

In [279]:
final_df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,22,20,1,10,2
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,5,1,2019,5,50,13,15,7
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,6,9,2019,9,25,4,25,19
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,5,12,2019,18,5,23,30,5
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,3,1,2019,16,50,21,35,4


In [280]:
# in some conditions the flight total travelling duration is 5m so m cant convert into int finally we remove those records
final_df["Duration_hours"]=final_df["Duration_hours"].astype("int64")

In [281]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13351 non-null  object 
 1   Source           13351 non-null  object 
 2   Destination      13351 non-null  object 
 3   Duration         13351 non-null  object 
 4   Total_Stops      13350 non-null  float64
 5   Additional_Info  13351 non-null  object 
 6   Price            10681 non-null  float64
 7   Date             13351 non-null  int64  
 8   Month            13351 non-null  int64  
 9   Year             13351 non-null  int64  
 10  Dep_Hours        13351 non-null  int64  
 11  Dep_Minutes      13351 non-null  int64  
 12  Arrival_Hours    13351 non-null  int64  
 13  Arrival_Minutes  13351 non-null  int64  
 14  Duration_hours   13351 non-null  int64  
dtypes: float64(2), int64(8), object(5)
memory usage: 1.6+ MB


In [282]:
final_df.drop("Duration",axis=1,inplace=True)
final_df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,IndiGo,Banglore,New Delhi,0.0,No info,3897.0,24,3,2019,22,20,1,10,2
1,Air India,Kolkata,Banglore,2.0,No info,7662.0,5,1,2019,5,50,13,15,7
2,Jet Airways,Delhi,Cochin,2.0,No info,13882.0,6,9,2019,9,25,4,25,19
3,IndiGo,Kolkata,Banglore,1.0,No info,6218.0,5,12,2019,18,5,23,30,5
4,IndiGo,Banglore,New Delhi,1.0,No info,13302.0,3,1,2019,16,50,21,35,4


In [283]:
#final_df.drop("hours",axis=1,inplace=True)
final_df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,IndiGo,Banglore,New Delhi,0.0,No info,3897.0,24,3,2019,22,20,1,10,2
1,Air India,Kolkata,Banglore,2.0,No info,7662.0,5,1,2019,5,50,13,15,7
2,Jet Airways,Delhi,Cochin,2.0,No info,13882.0,6,9,2019,9,25,4,25,19
3,IndiGo,Kolkata,Banglore,1.0,No info,6218.0,5,12,2019,18,5,23,30,5
4,IndiGo,Banglore,New Delhi,1.0,No info,13302.0,3,1,2019,16,50,21,35,4


In [284]:
# we move to categorical variables like airline source destination
final_df["Airline"].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [285]:
# using labelencoder to encodeing the categorical values 
from sklearn.preprocessing import LabelEncoder
# otherwise using OneHotEncoder

In [286]:
labelencoder = LabelEncoder()
final_df["Airline"] = labelencoder.fit_transform(final_df["Airline"])
final_df["Source"] = labelencoder.fit_transform(final_df["Source"])
final_df["Destination"] = labelencoder.fit_transform(final_df["Destination"])
final_df["Additional_Info"] = labelencoder.fit_transform(final_df["Additional_Info"])

In [287]:
final_df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,3,0,5,0.0,8,3897.0,24,3,2019,22,20,1,10,2
1,1,3,0,2.0,8,7662.0,5,1,2019,5,50,13,15,7
2,4,2,1,2.0,8,13882.0,6,9,2019,9,25,4,25,19
3,3,3,0,1.0,8,6218.0,5,12,2019,18,5,23,30,5
4,3,0,5,1.0,8,13302.0,3,1,2019,16,50,21,35,4


In [288]:
final_df.isnull().sum()

Airline               0
Source                0
Destination           0
Total_Stops           1
Additional_Info       0
Price              2670
Date                  0
Month                 0
Year                  0
Dep_Hours             0
Dep_Minutes           0
Arrival_Hours         0
Arrival_Minutes       0
Duration_hours        0
dtype: int64

In [289]:
final_df[final_df["Total_Stops"].isnull()]

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
9039,1,2,1,NaN,8,7480.0,5,6,2019,9,45,9,25,23


In [290]:
final_df.drop(9039,axis=0,inplace=True)
final_df

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,3,0,5,0.0,8,3897.0,24,3,2019,22,20,1,10,2
1,1,3,0,2.0,8,7662.0,5,1,2019,5,50,13,15,7
2,4,2,1,2.0,8,13882.0,6,9,2019,9,25,4,25,19
3,3,3,0,1.0,8,6218.0,5,12,2019,18,5,23,30,5
4,3,0,5,1.0,8,13302.0,3,1,2019,16,50,21,35,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,3,0,1.0,8,NaN,6,6,2019,20,30,20,25,23
2667,3,3,0,0.0,8,NaN,27,3,2019,14,20,16,55,2
2668,4,2,1,1.0,8,NaN,3,6,2019,21,50,4,25,6
2669,1,2,1,1.0,8,NaN,3,6,2019,4,0,19,15,15


In [291]:
final_df.isnull().sum()

Airline               0
Source                0
Destination           0
Total_Stops           0
Additional_Info       0
Price              2670
Date                  0
Month                 0
Year                  0
Dep_Hours             0
Dep_Minutes           0
Arrival_Hours         0
Arrival_Minutes       0
Duration_hours        0
dtype: int64

In [292]:
# it is not nessery to find because we split total_df into x and y x = normal features and y = prediction feature
test_data = final_df[final_df["Price"].isnull()]
test_data

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,4,2,1,1.0,8,NaN,6,6,2019,17,30,4,25,10
1,3,3,0,1.0,8,NaN,5,12,2019,6,20,10,20,4
2,4,2,1,1.0,5,NaN,21,5,2019,19,15,19,0,23
3,6,2,1,1.0,8,NaN,21,5,2019,8,0,21,0,13
4,0,0,2,0.0,8,NaN,24,6,2019,23,55,2,45,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,3,0,1.0,8,NaN,6,6,2019,20,30,20,25,23
2667,3,3,0,0.0,8,NaN,27,3,2019,14,20,16,55,2
2668,4,2,1,1.0,8,NaN,3,6,2019,21,50,4,25,6
2669,1,2,1,1.0,8,NaN,3,6,2019,4,0,19,15,15


In [293]:
# it is not nessery to find because we split total_df into x and y x = normal features and y = prediction feature
train_data = final_df[~final_df["Price"].isnull()]
train_data

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,3,0,5,0.0,8,3897.0,24,3,2019,22,20,1,10,2
1,1,3,0,2.0,8,7662.0,5,1,2019,5,50,13,15,7
2,4,2,1,2.0,8,13882.0,6,9,2019,9,25,4,25,19
3,3,3,0,1.0,8,6218.0,5,12,2019,18,5,23,30,5
4,3,0,5,1.0,8,13302.0,3,1,2019,16,50,21,35,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0,3,0,0.0,8,4107.0,4,9,2019,19,55,22,25,2
10679,1,3,0,0.0,8,4145.0,27,4,2019,20,45,23,20,2
10680,4,0,2,0.0,8,7229.0,27,4,2019,8,20,11,20,3
10681,10,0,5,0.0,8,12648.0,3,1,2019,11,30,14,10,2


In [294]:
Columns = ["Airline","Source","Destination","Total_Stops","Additional_Info","Date","Month","Year","Dep_Hours","Dep_Minutes","Arrival_Hours","Arrival_Minutes","Duration_hours"]
X = final_df.loc[:,Columns]

In [295]:
X.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
0,3,0,5,0.0,8,24,3,2019,22,20,1,10,2
1,1,3,0,2.0,8,5,1,2019,5,50,13,15,7
2,4,2,1,2.0,8,6,9,2019,9,25,4,25,19
3,3,3,0,1.0,8,5,12,2019,18,5,23,30,5
4,3,0,5,1.0,8,3,1,2019,16,50,21,35,4


In [296]:
Column = ["Price"]
Y = final_df.loc[:,Column]
Y.head()

,Price
0,3897.0
1,7662.0
2,13882.0
3,6218.0
4,13302.0


In [297]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X ,Y , test_size=0.33, random_state=42)

In [298]:
X_train.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
182,8,3,0,0.0,8,24,6,2019,22,20,0,40,2
10499,10,0,2,0.0,8,15,4,2019,19,30,22,15,2
292,4,3,0,1.0,5,6,6,2019,20,0,18,15,22
2837,4,2,1,1.0,8,27,3,2019,16,0,4,25,12
9766,0,2,1,1.0,8,24,6,2019,7,55,22,25,14


In [299]:
X_train.shape

(8944, 13)

In [300]:
y_train.shape

(8944, 1)

In [301]:
y_train.shape

(8944, 1)

In [302]:
y_train[y_train.isnull()==True]

,Price
182,NaN
10499,NaN
292,NaN
2837,NaN
9766,NaN
...,...
1284,NaN
5192,NaN
5391,NaN
860,NaN


In [303]:
y_train.mode()

,Price
0,10262.0


In [304]:
y_train.fillna(y_train.mean(),inplace=True)
y_train

,Price
182,9102.52895
10499,5613.00000
292,10844.00000
2837,12242.00000
9766,6152.00000
...,...
1284,9102.52895
5192,14151.00000
5391,12819.00000
860,6144.00000


In [305]:
y_train

,Price
182,9102.52895
10499,5613.00000
292,10844.00000
2837,12242.00000
9766,6152.00000
...,...
1284,9102.52895
5192,14151.00000
5391,12819.00000
860,6144.00000


In [306]:
X_test.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
8561,6,2,1,1.0,8,6,9,2019,10,0,1,30,15
7591,4,0,5,1.0,8,3,6,2019,14,5,9,0,18
8294,3,2,1,1.0,8,27,6,2019,10,35,1,30,14
1620,1,3,0,1.0,8,24,3,2019,9,25,11,10,25
3989,3,0,5,1.0,8,21,3,2019,7,30,12,20,4


In [307]:
y_test.head()

,Price
8561,13377.0
7591,17261.0
8294,5684.0
1620,NaN
3989,5694.0


In [308]:
from sklearn.tree import DecisionTreeRegressor

In [309]:
reg = DecisionTreeRegressor(max_depth=2,random_state=0)

In [310]:
reg.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=2, random_state=0)

In [311]:
reg.predict(X_test[0:10])

array([10256.19695837, 13950.00650865, 10256.19695837, 10256.19695837,
        5889.31297055, 13950.00650865, 10256.19695837, 10256.19695837,
       10256.19695837,  5889.31297055])

In [312]:
X_test.head(1)

,Airline,Source,Destination,Total_Stops,Additional_Info,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
8561,6,2,1,1.0,8,6,9,2019,10,0,1,30,15


In [313]:
X_test.head(2)

,Airline,Source,Destination,Total_Stops,Additional_Info,Date,Month,Year,Dep_Hours,Dep_Minutes,Arrival_Hours,Arrival_Minutes,Duration_hours
8561,6,2,1,1.0,8,6,9,2019,10,0,1,30,15
7591,4,0,5,1.0,8,3,6,2019,14,5,9,0,18


In [314]:
# Conclusion
# finally the roll no 8561 has the flight price is 10228.36030443

In [315]:
reg.predict(X_test.iloc[0].values.reshape(1,-1))

C:\Users\Purushoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([10256.19695837])

In [319]:
reg.predict(X_test.iloc[1].values.reshape(1,-1))

C:\Users\Purushoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([13950.00650865])

In [320]:
# for 8561 flight price 10228.36030443
# for 7591 flight price 13894.31967213

In [318]:
reg.predict(X_test.iloc[2].values.reshape(1,-1))

C:\Users\Purushoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([10256.19695837])

In [ ]:
# for 8294 flight price 10228.36030443